<a href="https://colab.research.google.com/github/Samkyang/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backpor

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...| 2013-01-14|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...|         Apparel|          5|    

In [ ]:
# filter DataFrame for total_votes above or equal to 20
tv_df = df.filter(df.total_votes >= 20)
tv_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|   26252807|R2P76PJFUGXBHO|B013S8HI5U|      22756004|Lovenest Men's- S...|         Apparel|          1|    

In [ ]:
# Filter DataFrame for helpful_votes ratio above or equal to 50%
HV_DF = tv_df.filter("helpful_votes/total_votes>=.50")
HV_DF.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|     453792| RQW4AFOG9MR4Z|B013QTWRAC|     254612789|UONBOX Celebrity ...|         Apparel|          5|    

In [ ]:
# Create paid vine DataFrame
paid_df = HV_DF.filter(HV_DF.vine == 'Y')
paid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   51744990| R6U9701C3BGO6|B00XL2X3U8|     139556466|Wrangler Authenti...|         Apparel|          3|          139|        147|   Y|                N|        Comfort-Lite|After some debate...| 2015-08-31|
|         US|   35360512|R1XK3ALB45D7N4|B00XKYRLEQ|     243221870|Wrangler Authenti...|         Apparel|          5|    

In [53]:
# Create non-paid vine DataFrame
npaid_df = HV_DF.filter(HV_DF.vine == 'N')
npaid_df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...| 2015-08-28|
|         US|     453792| RQW4AFOG9MR4Z|B013QTWRAC|     254612789|UONBOX Celebrity ...|         Apparel|          5|    

In [ ]:
# Total number of ALL reviews
total_reviews_count = df.count()
total_reviews_count

5906333

In [ ]:
#Total number of ALL 5-star reviews

FiveStar_df = df.filter(df["star_rating"] == '5').count()
FiveStar_df

3320557

In [ ]:
# Percentage of ALL 5-star reviews non-paid 

0.013668791109443386

In [51]:
# Number of paid 5 star reviews
paid_df_star = paid_df.filter(paid_df.star_rating == 5).count()
paid_df_star

23733

In [54]:
# Number of non-paid 5 star reviews
npaid_df_star = npaid_df.filter(npaid_df.star_rating == 5).count()
npaid_df_star

23733

In [60]:
# Percentage of paid 5 star reviews
PPFR = (paid_df_star/total_reviews_count)*100
PPFR

0.4018229246471542

In [61]:
# Percentage of non-paid 5 star reviews
PNFR = (npaid_df_star/total_reviews_count)*100
PNFR

0.4018229246471542